# Webscraping : Equipes
Récupération des données portant sur les statistiques d'équipes de NBA entre la saison 2005-2006 et la saison 2018-2019.
Données en libre accès sur le site officiel de la NBA.

URL : https://www.nba.com/stats/teams/traditional/?sort=W_PCT&dir=-1

In [4]:
# Import de bibliothèques
import bs4
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from time import sleep

In [5]:
# Création de la liste de mois et d'années que l'on veut récupérer
months=['October', 'November', 'December', 'January', 'February', 'March', 'April']
months_2=[10,11,12,1,2,3,4]
years=[2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2018]
list_date=[]
for i in years:
    for k in months_2:
        if k in [10,11,12]:
            list_date.append(str([i,k]))
        if k in [1,2,3,4]:
            list_date.append(str([i+1,k]))
print(list_date)
seasons=['2005-06','2006-07','2007-08','2008-09','2009-10','2010-11','2011-12','2012-13',
         '2013-14','2014-15','2015-16','2016-17','2017-18','2018-19']

['[2005, 10]', '[2005, 11]', '[2005, 12]', '[2006, 1]', '[2006, 2]', '[2006, 3]', '[2006, 4]', '[2006, 10]', '[2006, 11]', '[2006, 12]', '[2007, 1]', '[2007, 2]', '[2007, 3]', '[2007, 4]', '[2007, 10]', '[2007, 11]', '[2007, 12]', '[2008, 1]', '[2008, 2]', '[2008, 3]', '[2008, 4]', '[2008, 10]', '[2008, 11]', '[2008, 12]', '[2009, 1]', '[2009, 2]', '[2009, 3]', '[2009, 4]', '[2009, 10]', '[2009, 11]', '[2009, 12]', '[2010, 1]', '[2010, 2]', '[2010, 3]', '[2010, 4]', '[2010, 10]', '[2010, 11]', '[2010, 12]', '[2011, 1]', '[2011, 2]', '[2011, 3]', '[2011, 4]', '[2011, 10]', '[2011, 11]', '[2011, 12]', '[2012, 1]', '[2012, 2]', '[2012, 3]', '[2012, 4]', '[2012, 10]', '[2012, 11]', '[2012, 12]', '[2013, 1]', '[2013, 2]', '[2013, 3]', '[2013, 4]', '[2013, 10]', '[2013, 11]', '[2013, 12]', '[2014, 1]', '[2014, 2]', '[2014, 3]', '[2014, 4]', '[2014, 10]', '[2014, 11]', '[2014, 12]', '[2015, 1]', '[2015, 2]', '[2015, 3]', '[2015, 4]', '[2015, 10]', '[2015, 11]', '[2015, 12]', '[2016, 1]', '[20

In [7]:
# Liste d'équipes lors de la dernière saison scrapée
list_teams=['Atlanta Hawks',
 'Boston Celtics',
 'Brooklyn Nets',
 'Charlotte Hornets',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Houston Rockets',
 'Indiana Pacers',
 'LA Clippers',
 'Los Angeles Lakers',
 'Memphis Grizzlies',
 'Miami Heat',
 'Milwaukee Bucks',
 'Minnesota Timberwolves',
 'New Orleans Pelicans',
 'New York Knicks',
 'Oklahoma City Thunder',
 'Orlando Magic',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Portland Trail Blazers',
 'Sacramento Kings',
 'San Antonio Spurs',
 'Toronto Raptors',
 'Utah Jazz',
 'Washington Wizards']

In [8]:
# Fonction pour récupérer les données d'équipes d'un mois sur le site de la NBA avec Selenium et les placer dans 
# un Dataframe
def scrap_teams(season,month):
    url='https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season='+ season + '&SeasonType=Regular%20Season&Month=' + str(month)
    print(url)
    DRIVER_PATH = 'chromedriver'
    driver = webdriver.Chrome(executable_path=DRIVER_PATH)
    driver.get(url)
    driver.implicitly_wait(10)
    sleep(5)
    # Si les données sont disponibles, la fonction renvoie le dataframe, sinon la fonction renvoie None
    try:
        h3 = driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div/div/nba-stat-table')
        text = h3.text
        list_1=text.split('\n')
        index = list_1.index('TEAM')
        del list_1[index:]
        header=list_1.pop(0).split(' ')
        dataframe_list=[]
        i=0
        for k in range(len(list_1)):
            #print(k)
            if i==1:
                chiffres=list_1[k].split(' ')
                stat_equipe=[list_1[k-1]] + chiffres
                #print(stat_equipe)
                dataframe_list.append(stat_equipe)
                #print('dataframe', dataframe_list)
            i+=1
            if i==2:
                i=0
        df = pd.DataFrame(dataframe_list, columns = header)
        # Dans le Dataframe obtenu, si une équipe a encore son ancien nom, on le remplace par le dernier nom connu 
        # pour avoir les 30 mêmes équipes dans chaque saison.
        teams = df['TEAM'].tolist()
        if 'New Jersey Nets' in teams:
            teams = list(map(lambda x: x.replace('New Jersey Nets', 'Brooklyn Nets'), teams))
        if 'Charlotte Bobcats' in teams:
            teams = list(map(lambda x: x.replace('Charlotte Bobcats', 'Charlotte Hornets'), teams))
        if 'New Orleans/Oklahoma City Hornets' in teams:
            teams = list(map(lambda x: x.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans'), teams))
        if 'New Orleans Hornets' in teams:
            teams = list(map(lambda x: x.replace('New Orleans Hornets', 'New Orleans Pelicans'), teams))
        if 'Seattle SuperSonics' in teams:
            teams = list(map(lambda x: x.replace('Seattle SuperSonics', 'Oklahoma City Thunder'), teams))
        if 'Los Angeles Clippers' in teams:
            teams = list(map(lambda x: x.replace('Los Angeles Clippers', 'LA Clippers'), teams))           
        df['TEAM']=teams
        df=df.set_index('TEAM')
        df = df.sort_index()
        return df
    except:
        return None

In [47]:
# Cellule réalisant le webscraping pour toutes les saisons et mois voulus
date_n=0
df_final=pd.DataFrame(columns=list_teams)

for season in seasons:
    for month in range(1,8):
        print(season + '  ---  ' + str(month))
        date=list_date[date_n]
        print(date)
        df_mois=scrap_teams(season,month)
        # Si la fonction renvoie bien un Dataframe et pas None, le programme poursuit le traitement du fichier. Sinon
        # on ajoute une ligne de NaN correspondant au mois pour lequel les données ne sont pas disponibles.
        if isinstance(df_mois, pd.DataFrame):
            list_teams_2=df_mois.index.tolist()
            list_teams_2.sort()
            # On vérifie que dans le Dataframe des stats du mois, les équipes sont les mêmes que dans la liste 
            # initiale des équipes afin qu'il y ait bien 30 colonnes dans le Dataframe final correspondant à 
            # chaque équipe.
            if list_teams_2 == list_teams:
                print("Mêmes équipes")
                values=[]
                for team in list_teams:
                    df2=df_mois.loc[[team]]
                    values.append(df2.to_dict('records')[0])
                    #print(values)
                df_final.loc[date]=values
            elif all(elem in list_teams for elem in list_teams_2) and list_teams_2 != list_teams:
                print("Toutes les équipes n'ont pas participé ce mois-ci")
                values=[]
                for team in list_teams:
                    if team in list_teams_2:
                        df2=df_mois.loc[[team]]
                        values.append(df2.to_dict('records')[0])                    
                    else:
                        values.append(np.nan)
                df_final.loc[date]=values
            else:
                print("---------------- ERREUR -----------------------")           
        else:
            print('Données pas dispos')
            df_final.loc[date]=[np.NAN]*30
        date_n+=1
print(df_final)

2005-06  ---  1
[2005, 10]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=1


<ipython-input-46-72aef711ecbe>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  h3 = driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div/div/nba-stat-table')


Données pas dispos
2005-06  ---  2
[2005, 11]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=2
Mêmes équipes
2005-06  ---  3
[2005, 12]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=3
Mêmes équipes
2005-06  ---  4
[2006, 1]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=4
Mêmes équipes
2005-06  ---  5
[2006, 2]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=5
Mêmes équipes
2005-06  ---  6
[2006, 3]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=6
Mêmes équipes
2005-06  ---  7
[2006, 4]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2005-06&SeasonType=Regular%20Season&Month=7
Mêmes équipes
2006-07  ---  1
[2006, 10]
http

Mêmes équipes
2012-13  ---  4
[2013, 1]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2012-13&SeasonType=Regular%20Season&Month=4
Mêmes équipes
2012-13  ---  5
[2013, 2]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2012-13&SeasonType=Regular%20Season&Month=5
Mêmes équipes
2012-13  ---  6
[2013, 3]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2012-13&SeasonType=Regular%20Season&Month=6
Mêmes équipes
2012-13  ---  7
[2013, 4]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2012-13&SeasonType=Regular%20Season&Month=7
Mêmes équipes
2013-14  ---  1
[2013, 10]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2013-14&SeasonType=Regular%20Season&Month=1
Mêmes équipes
2013-14  ---  2
[2013, 11]
https://www.nba.com/stats/teams/traditional/?sort=TEAM_NAME&dir=-1&Season=2013-14&SeasonType=Regular%20Season&Month=2
Mêmes équipes
2013-14  ---  3
[2013, 12]
https://w

In [49]:
# Le Dataframe final est transformé en fichier .csv
df_final.to_csv('stats-equipes-par-mois.csv', sep=';')